# Lab 6: Collaborative Filtering

In this lab, we will walk through an implementation of collaborative filtering in Python. After seeing a complete implementation, you will then implement a couple of simpler approaches (called baselines) for comparison. 

This lab is adapted from the following online tutorial: https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/ 

We will be using the MovieLens dataset. This is a classic dataset for training recommendation models. There are various datasets, but the one that we will use below consists of 100,000 movie ratings by users (on a 1-5 scale). The main data file consists of a tab-separated list with user-id (starting at 1), item-id (starting at 1), rating, and timestamp as the four fields. 

In [61]:
import numpy as np
import pandas as pd

First we will read in the dataset and get some basic stats from the data.

In [62]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [63]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print (str(n_users) + ' users')
print (str(n_items) + ' items')

943 users
1682 items


Most recommendation models consist of building a user-by-item matrix with some sort of "interaction" number in each cell. If one includes the numerical ratings that users give items, then this is called an *explicit feedback* model. Alternatively, one may include *implicit feedback* which are actions by a user that signify a positive or negative preference for a given item (such as viewing the item online). These two scenarios often must be treated differently.

In the case of the MovieLens dataset, we have ratings, so we will focus on explicit feedback models. First, we must construct our user-item matrix. We can easily map user/item ID's to user/item indices by removing the "Python starts at 0" offset between them.

In [64]:
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
ratings

array([[ 5.,  3.,  4., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.]])

In this dataset, every user has rated at least 20 movies which results in a reasonable sparsity of 6.3%. This means that 6.3% of the user-item ratings have a value. Note that, although we filled in missing ratings as 0, we should not assume these values to truly be zero. More appropriately, they are just empty entries. 

We will split our data into training and test sets by removing 10 ratings per user from the training set and placing them in the test set.

In [65]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

In [66]:
train, test = train_test_split(ratings)

## Collaborative filtering
We will focus on collaborative filtering models today which can be generally split into two classes: user- and item-based collaborative filtering. In either scenario, one builds a similarity matrix. For user-based collaborative filtering, the user-similarity matrix will consist of some distance metric that measures the similarity between any two pairs of users. Likewise, the item-similarity matrix will measure the similarity between any two pairs of items. 

__Note:__ The original tutorial covers both user-based and item-based implementations, but in this lab we focus on just the user-based approach.

A common distance metric is cosine similarity. The metric can be thought of geometrically if one treats a given user's (item's) row (column) of the ratings matrix as a vector. For user-based collaborative filtering, two users' similarity is measured as the cosine of the angle between the two users' vectors. For users ${u}$ and ${u'}$, the cosine similarity is

$$
sim(u, u') = 
cos(\theta{}) = 
\frac{\textbf{r}_{u} \dot{} \textbf{r}_{u'}}{\| \textbf{r}_{u} \| \| \textbf{r}_{u'} \|} = 
\sum_{i} \frac{r_{ui}r_{u'i}}{\sqrt{\sum\limits_{i} r_{ui}^2} \sqrt{\sum\limits_{i} r_{u'i}^2} }
$$

This can be written as a for-loop with code, but the Python code will run quite slow; instead, one should try to express any equation in terms of NumPy functions. I've included a slow and fast version of the cosine similarity function below. The slow function took so long that I eventually canceled it because I got tired of waiting. The fast function, on the other hand, takes around 200 ms. 

The cosine similarity will range from 0 to 1 in our case (because there are no negative ratings). Notice that it is symmetric and has ones along the diagonal.

In [67]:
def slow_similarity(ratings):
    axmax = 0
    axmin = 1

    sim = np.zeros((ratings.shape[axmax], ratings.shape[axmax]))
    for u in range(ratings.shape[axmax]):
        for uprime in range(ratings.shape[axmax]):
            rui_sqrd = 0.
            ruprimei_sqrd = 0.
            for i in range(ratings.shape[axmin]):
                sim[u, uprime] = ratings[u, i] * ratings[uprime, i]
                rui_sqrd += ratings[u, i] ** 2
                ruprimei_sqrd += ratings[uprime, i] ** 2
            sim[u, uprime] /= rui_sqrd * ruprimei_sqrd
    return sim

def fast_similarity(ratings, epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    sim = ratings.dot(ratings.T) + epsilon   
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

   

In [68]:
#%timeit slow_user_similarity(train)

In [69]:
%timeit fast_similarity(train)

83.4 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [70]:
user_similarity = fast_similarity(train)
print(user_similarity[:4, :4])

[[ 1.          0.12757633  0.0396659   0.06130388]
 [ 0.12757633  1.          0.11743494  0.21153424]
 [ 0.0396659   0.11743494  1.          0.26018616]
 [ 0.06130388  0.21153424  0.26018616  1.        ]]


With our similarity matrix in hand, we can now predict the ratings that were not included with the data. Using these predictions, we can then compare them with the test data to attempt to validate the quality of our recommender model.

For user-based collaborative filtering, we predict that a user's $u$'s rating for item $i$ is given by the weighted sum of all other users' ratings for item $i$ where the weighting is the cosine similarity between the each user and the input user $u$.

$$\hat{r}_{ui} = \sum\limits_{u'}sim(u, u') r_{u'i}$$

We must also normalize by the number of ${r_{u'i}}$ ratings:

$$\hat{r}_{ui} = \frac{\sum\limits_{u'} sim(u, u') r_{u'i}}{\sum\limits_{u'}|sim(u, u')|}$$

As with before, our computational speed will benefit greatly by favoring NumPy functions over for loops. With our slow function below, even though I use NumPy methods, the presence of the for-loop still slows the algorithm


In [71]:
def predict_slow_simple(ratings, similarity):
    pred = np.zeros(ratings.shape)

    for i in range(ratings.shape[0]):
        for j in range(ratings.shape[1]):
            pred[i, j] = similarity[i, :].dot(ratings[:, j])\
                         /np.sum(np.abs(similarity[i, :]))
    return pred

def predict_fast_simple(ratings, similarity):
    return similarity.dot(ratings) / np.array([np.abs(similarity).sum(axis=1)]).T
    

In [72]:
#%timeit predict_slow_simple(train, user_similarity, kind='user')

In [73]:
%timeit predict_fast_simple(train, user_similarity)

124 ms ± 3.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


We'll use the scikit-learn's mean squared error function as our validation metric. 

In [74]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [75]:
user_prediction = predict_fast_simple(train, user_similarity)

print('User-based CF MSE: ' + str(get_mse(user_prediction, test)))


User-based CF MSE: 8.38595260368


You should see an MSE score of around 8.42. __Note:__ This will vary slightly on each run because of randomness in how the train/test split is created.

# Lab Assignment

For the lab assignment, you need to complete a few additional steps that will help us understand the performance of the collaborative filtering system. If we just see a score of 8.42, it is hard to know if that is bad or good. You will implement a couple of simple baseline models for comparison. A baseline is normally something that is very simple to implement but which might be expected to work well. Some baselines can be surprisingly hard to beat on certain tasks. 

__Baseline 1__: For this baseline, you just need to calculate what the MSE would be if we did not do anything. That is, what would the MSE be if we just left the zeroes in the training set and compared with the test set? This should be very easy to implement, and if you implement it correctly you should see an MSE around 14.2. __Note:__ The MSE will vary on each run because there is randomness in how the train/test split is determined.  

In [120]:
# Your Baseline 1 code here
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)
    

user_prediction = predict_fast_simple(train, user_similarity)

print('User-based CF MSE: ' + str(get_mse(train, test)))

User-based CF MSE: 14.1205726405


The next baselines are also simple, but will take a few more lines of code to implement. They use something called _median imputation_. Basically, for every 0 in the training set, we will replace it with the median (central) value. There are two ways to do this. If a particular user/movie combo in the dataset has a 0, we could replace it with either:
* the median value of all the user's ratings
* the median value of all the movie's ratings

You will implement both of those approaches. One of them calculates a median across rows, and the other calculates a median across columns. 

Pandas has some useful built-in functions for doing median imputation (and other types of imputation), so your best bet is to convert the training data to a Pandas DataFrame. And those Pandas functions are designed to replace 'NaN' values, so the zeroes need to be converted to 'NaN' values. 

We will walk through a __toy example__ below that you can use as the basis of your implementations.

In [84]:
# some toy data, with 0 (missing) values
X = np.array([[0, 1, 2], [1,0,3], [0, 2, 2], [3,0,0]])        
print(X)



[[0 1 2]
 [1 0 3]
 [0 2 2]
 [3 0 0]]


In [78]:
# convert to Pandas DataFrame
dat = pd.DataFrame(X)
# replace zeroes with 'NaN' values
dat = dat.replace(0, np.nan)
# use the fillna() function to replace 'NaN' values with the median (over rows).
# if axis=0, then it calculates the median by columns. 
dat = dat.fillna(dat.median(axis=1))

# if there is some column (or row) that has only 'NaN' values,
# then the median cannot be calculated and nothing will be imputed. 
# we need to ensure all 'NaN' values are replaced, so we could change 
# the remaining ones to 0 (or do something else like use the median over the entire dataset). 

dat = dat.replace(np.nan, 0)

# then convert back to a NumPy array
impute = dat.values
print(impute)

[[ 1.5  1.   2. ]
 [ 1.   2.   3. ]
 [ 1.5  2.   2. ]
 [ 3.   2.   2. ]]


__Baseline2 :__ Do median imputation over columns (movies) and calculate the resulting MSE. 

In [157]:
# Your Baseline 2 code here
dat = pd.DataFrame(train)
dat = dat.replace(0, np.nan)
dat = dat.fillna(dat.median(axis=1))
dat = dat.replace(np.nan, 0)

impute = dat.values
print(impute)

print('User-based CF MSE: ' + str(get_mse(test, user_prediction)))

[[ 5.  3.  4. ...,  0.  0.  0.]
 [ 4.  4.  3. ...,  0.  0.  0.]
 [ 4.  4.  3. ...,  0.  0.  0.]
 ..., 
 [ 4.  4.  3. ...,  0.  0.  0.]
 [ 4.  4.  3. ...,  0.  0.  0.]
 [ 4.  5.  3. ...,  0.  0.  0.]]
User-based CF MSE: 0.249129400157


__Baseline 3:__ Do median imputation over rows (users) and calculate the resulting MSE. 

In [153]:
# Your Baseline 3 code here
dat = pd.DataFrame(train)
dat = dat.replace(0, np.nan)
dat = dat.fillna(dat.median(axis=0))
dat = dat.replace(np.nan, 0)

impute = dat.values
print(impute)

print('User-based CF MSE: ' + str(get_mse(train, user_similarity)))

[[ 5.  3.  4. ...,  2.  3.  3.]
 [ 4.  3.  3. ...,  2.  3.  3.]
 [ 4.  3.  3. ...,  2.  3.  3.]
 ..., 
 [ 4.  3.  3. ...,  2.  3.  3.]
 [ 4.  3.  3. ...,  2.  3.  3.]
 [ 4.  5.  3. ...,  2.  3.  3.]]
User-based CF MSE: 1.19950620084


If you implement median imputation correctly, you should see the MSE on the test set is actually considerably lower than when using collaborative filtering. The MSE scores will be around 2.4 and 1.2, respectively, for the median imputation approaches. 

This does not mean that the collaborative filtering approach is bad. But we always need to analyse systems in the context of baselines and performance bounds. In this case, a simple approach works very well. It might be possible to combine collaborative filtering and median imputation to get even better performance (good project idea). 

__Optional:__ If you finish the above steps very quickly, try some additional approaches:
* Use mean imputation instead of median imputation. What is the MSE?
* Try imputing the median or mean over the entire dataset (rather than column-by-column or row-by-row). What is the MSE?


__Deliverables:__ Submit your completed notebook via Blackboard.